### 1. [20 pts] Download the datasets from the course website, unzip (7z program) and load it with pandas.

### (Hint: df = pd.read_json("..json", orient='records', lines=True)

In [2]:
import pandas as pd
import re 

In [3]:
import pandas as pd

df_train = pd.read_json("arxiv_training.json", orient='records', lines=True)
df_test = pd.read_json("arxiv_testing.json", orient='records', lines=True)

In [4]:
# Then, filter out the rows where the year is after 2010
df_test['update_date'] = pd.to_datetime(df_test['update_date'])
df_train['update_date'] = pd.to_datetime(df_train['update_date'])


filtered_df_test = df_test[df_test['update_date'].dt.year >= 2010]
filtered_df_train = df_train[df_train['update_date'].dt.year >= 2010]

### Use the following site from arxiv (https://arxiv.org/category_taxonomy) to taxonomize the documents into 5 categories: {0:'Computer Science', 1:'Mathematics', 2:'Statistics', 3:'Economics', 4:'EESS'}. This will constitute the ground truth. To create labels, use the first category entry.
### (Hint: Use the regular expression re.search(r'^([\w\-]+)', cat.split(' ')[0]).group(1) on the categories column. Check: [110347, 130726, 13920, 924, 7252] and [111091, 130287, 13672, 871, 7249] in training and testing sizes, respectively)

In [5]:
def sanitize(cat):
    sant=re.search(r'^([\w\-]+)', cat.split(' ')[0]).group(1)
    five_cats={'cs','stat','math','econ','eess'}
    if sant in five_cats:
        return sant
    else:
        return 'NA'
filtered_df_test['new_cat']=filtered_df_test['categories'].apply(sanitize).copy()
filtered_df_train['new_cat']=filtered_df_train['categories'].apply(sanitize).copy()

/tmp/ipykernel_1704/3604868860.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_test['new_cat']=filtered_df_test['categories'].apply(sanitize).copy()
/tmp/ipykernel_1704/3604868860.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_train['new_cat']=filtered_df_train['categories'].apply(sanitize).copy()


In [6]:
from collections import Counter
cats=['cs','math','stat','econ','eess']
train=[110347, 130726, 13920, 924, 7252] 
test=[111091, 130287, 13672, 871, 7249]
train_check=Counter(filtered_df_train['new_cat'])
test_check=Counter(filtered_df_test['new_cat'])
for i,category in enumerate(cats):
    print(category,'train',train_check[category],train[i])
    print(category,'test',test_check[category],test[i])

    

cs train 115106 110347
cs test 116397 111091
math train 149465 130726
math test 148482 130287
stat train 14305 13920
stat test 14251 13672
econ train 903 924
econ test 892 871
eess train 7285 7252
eess test 7217 7249


In [7]:
filtered_df_train[filtered_df_train['new_cat'] == 'econ']

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,new_cat
427,2007.10269,David Roodman,David Roodman,The domestic economic impacts of immigration,None,None,None,None,econ.GN q-fin.EC,http://creativecommons.org/licenses/by-nc-sa/4.0/,This paper critically reviews the research o...,"[{'version': 'v1', 'created': 'Thu, 16 Jul 202...",2020-07-21,"[[Roodman, David, ]]",econ
598,1905.02917,Federico Echenique,Christopher P. Chambers and Federico Echenique,Spherical Preferences,None,None,None,None,econ.TH,http://arxiv.org/licenses/nonexclusive-distrib...,We introduce and study the property of ortho...,"[{'version': 'v1', 'created': 'Wed, 8 May 2019...",2020-02-14,"[[Chambers, Christopher P., ], [Echenique, Fed...",econ
670,1808.10428,Andrea Zaccaria,"Angelica Sbardella, Emanuele Pugliese, Andrea ...",The role of complex analysis in modeling econo...,None,None,10.3390/e20110883,None,econ.GN q-fin.EC,http://arxiv.org/licenses/nonexclusive-distrib...,Development and growth are complex and tumul...,"[{'version': 'v1', 'created': 'Thu, 30 Aug 201...",2018-12-05,"[[Sbardella, Angelica, ], [Pugliese, Emanuele,...",econ
692,1908.04336,Federico Echenique,Federico Echenique and Antonio Miralles and Ju...,Fairness and efficiency for probabilistic allo...,None,None,None,None,econ.TH cs.GT,http://arxiv.org/licenses/nonexclusive-distrib...,We propose a notion of fairness for allocati...,"[{'version': 'v1', 'created': 'Mon, 12 Aug 201...",2020-05-12,"[[Echenique, Federico, ], [Miralles, Antonio, ...",econ
1062,1908.01103,Gunduz Caginalp,Carey Caginalp and Gunduz Caginalp,Derivation of non-classical stochastic price d...,None,None,None,None,econ.TH math.PR,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze the relative price change of asse...,"[{'version': 'v1', 'created': 'Sat, 3 Aug 2019...",2020-08-26,"[[Caginalp, Carey, ], [Caginalp, Gunduz, ]]",econ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327907,1710.01423,Chuan Goh,Chuan Goh,Rate-Optimal Estimation of the Intercept in a ...,None,None,None,None,econ.EM,http://arxiv.org/licenses/nonexclusive-distrib...,This paper presents a new estimator of the i...,"[{'version': 'v1', 'created': 'Wed, 4 Oct 2017...",2018-09-26,"[[Goh, Chuan, ]]",econ
328410,1805.03275,Juan Carlos Escanciano,Juan Carlos Escanciano and Wei Li,Optimal Linear Instrumental Variables Approxim...,None,None,None,None,econ.EM,http://arxiv.org/licenses/nonexclusive-distrib...,This paper studies the identification and es...,"[{'version': 'v1', 'created': 'Tue, 8 May 2018...",2020-02-06,"[[Escanciano, Juan Carlos, ], [Li, Wei, ]]",econ
328509,1910.11337,V\'itor V. Vasconcelos,"V\'itor V. Vasconcelos, Phillip M. Hannam, Sim...",Coalition-structured governance improves coope...,16 pages (2 figures) plus supplementary materi...,None,None,None,econ.GN nlin.AO q-fin.EC,http://creativecommons.org/licenses/by/4.0/,While the benefits of common and public good...,"[{'version': 'v1', 'created': 'Thu, 24 Oct 201...",2019-10-28,"[[Vasconcelos, Vítor V., ], [Hannam, Phillip M...",econ
328524,2008.07650,Ryan Kostiuk,Ryan Steven Kostiuk,Mobility and Social Efficiency,None,None,None,None,econ.TH,http://creativecommons.org/licenses/by-nc-sa/4.0/,This is a general competitive analysis paper...,"[{'version': 'v1', 'created': 'Mon, 17 Aug 202...",2020-08-19,"[[Kostiuk, Ryan Steven, ]]",econ


In [8]:
filtered_df_test.loc['categories' ]

KeyError: 'categories'